In [27]:
!pip install pandas
!pip install requests
!pip install tqdm


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import pandas as pd
import os
import requests
import time
import re

In [29]:
import re

def extract_year_period_week(filename):
    """Extracts year, period, and week from a filename like '2024_p1_w1.csv'.

    Args:
        filename: The name of the file.

    Returns:
        A tuple containing the year, period, and week as integers.
    """
    match = re.search(r"(\d+)_p(\d+)_w(\d+)", filename)
    if match:
        year = str(match.group(1))
        period = int(match.group(2))
        week = int(match.group(3))
        return year, period, week
    else:
        return None, None, None  # Handle cases where the pattern is not found

In [30]:
import pandas as pd

for filename in ['data/2024_p1_w1.csv']:
    df = pd.read_csv(filename, dtype={'GROWER': str, 'TERRAIN': str})
    year, period, week = extract_year_period_week(filename)

    if year is not None and period is not None and week is not None:
        df['year'] = year  # Add 'Year' column
        df['period'] = period  # Add 'Period' column
        df['week'] = week  # Add 'Week' column
    else:
        print(f"Warning: Could not extract year, period, and week from {filename}")

    # Process the DataFrame further...

In [31]:
df.head()

,GROWER,TERRAIN,WK_10,WK_11,WK_12,WK_13,WK_14,WK_15,WK_16,WK_17,...,STEMS,KLS.A,BXS.A,KLS.B,BXS.B,startdate,enddate,year,period,week
0,A. NOEL,0304,0,0,18,68,33,12,1,0,...,85,1180.87,88.12,59.57,4.45,25-03-2024,30-03-2024,2024,1,1
1,RABIA,0304,0,0,7,9,2,1,0,0,...,8,134.45,10.03,9.60,0.72,25-03-2024,30-03-2024,2024,1,1
2,"ALVAREZ, E.",0006,0,0,45,42,53,17,12,0,...,186,2746.25,204.95,58.18,4.34,25-03-2024,30-03-2024,2024,1,1
3,"ONGKINGCO, J.",0013,0,0,8,36,53,21,2,0,...,138,2288.52,170.78,21.40,1.60,25-03-2024,30-03-2024,2024,1,1
4,"REYES, P.",0027,0,0,6,0,82,20,1,0,...,88,1372.31,102.44,18.48,1.38,25-03-2024,30-03-2024,2024,1,1


In [32]:
melted_df = pd.melt(df,
                   id_vars=['GROWER',
                             'TERRAIN',
                             'BAGGING',
                             'STEMS',
                             'KLS.A',
                             'BXS.A',
                             'KLS.B',
                             'BXS.B',
                            'startdate','enddate','year','period','week'],
                   value_vars=['WK_10', 'WK_11', 'WK_12', 'WK_13',
                               'WK_14', 'WK_15', 'WK_16', 'WK_17'],
                   var_name='Week_Name',
                   value_name='stem_harvest_week_count')

display(melted_df.head())

,GROWER,TERRAIN,BAGGING,STEMS,KLS.A,BXS.A,KLS.B,BXS.B,startdate,enddate,year,period,week,Week_Name,stem_harvest_week_count
0,A. NOEL,0304,198,85,1180.87,88.12,59.57,4.45,25-03-2024,30-03-2024,2024,1,1,WK_10,0
1,RABIA,0304,73,8,134.45,10.03,9.60,0.72,25-03-2024,30-03-2024,2024,1,1,WK_10,0
2,"ALVAREZ, E.",0006,176,186,2746.25,204.95,58.18,4.34,25-03-2024,30-03-2024,2024,1,1,WK_10,0
3,"ONGKINGCO, J.",0013,129,138,2288.52,170.78,21.40,1.60,25-03-2024,30-03-2024,2024,1,1,WK_10,0
4,"REYES, P.",0027,96,88,1372.31,102.44,18.48,1.38,25-03-2024,30-03-2024,2024,1,1,WK_10,0


In [33]:
def rename_csv_files(directory):
  """Renames CSV files in a directory from 'p{period}_w{week}.csv' to 'processed_p{period}_w{week}.csv'.

  Args:
    directory: The directory containing the CSV files.
  """
  for filename in os.listdir(directory):
    if filename.endswith(".csv") and filename.startswith("p"):
      base_name = os.path.splitext(filename)[0]  # Get filename without extension
      period, week = base_name[1:].split("_")  # Extract period and week
      new_filename = f"processed_{period}_{week}.csv"
      os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
      print(f"Renamed '{filename}' to '{new_filename}'")

In [34]:
import requests
import time
import json
from datetime import datetime
from tqdm import tqdm  # Import tqdm for progress bar

# Function to convert date format
def format_date(date_str):
    try:
        return datetime.strptime(date_str, "%d-%m-%Y").strftime("%Y-%m-%d")
    except ValueError:
        print(f"Invalid date format: {date_str}")
        return None  

# List of Strapi API endpoints
strapi_api_endpoints = [
    "http://127.0.0.1:1337/api/harvests"
]

# Get total rows for progress bar
total_rows = len(melted_df)

# Iterate through rows with tqdm progress bar
for index, row in tqdm(melted_df.iterrows(), total=total_rows, desc="Processing Rows", unit="row"):
    formatted_startdate = format_date(str(row["startdate"]))
    formatted_enddate = format_date(str(row["enddate"]))

    data = {
        "data": {
            "grower": str(row["GROWER"]),
            "terrainid": str(row["TERRAIN"]),
            "bagging": int(row["BAGGING"]),
            "stems": int(row["STEMS"]),
            "class_type_a": float(row["KLS.A"]),
            "box_type_a": float(row["BXS.A"]),
            "class_type_b": float(row["KLS.B"]),
            "box_type_b": float(row["BXS.B"]),                        
            "stem_harvest_week": str(row["Week_Name"]),
            "stem_harvest_count": int(row["stem_harvest_week_count"]),
            "startdate": formatted_startdate,
            "enddate": formatted_enddate,
            "period": int(row["period"]),
            "week": int(row["week"]),
            "year": str(row["year"]),
        }
    }

    for endpoint in strapi_api_endpoints:
        try:
            response = requests.post(endpoint, json=data)
            response.raise_for_status()  
        except requests.exceptions.RequestException as e:
            print(f"\nError posting to {endpoint} at row {index}: {e}")

    time.sleep(0.3)  # Small delay to avoid overwhelming the server


Processing Rows:   0%|          | 0/192 [00:00<?, ?row/s]

Processing Rows: 100%|██████████| 192/192 [01:05<00:00,  2.92row/s]


In [ ]:
# prompt: from the melted_df save it to a csv
new_filename = f'processed_{file_name.split('_')[0][1]}_{file_name.split('_')[1].split('.')[0][1]}.csv'
melted_df.to_csv(new_filename, index=False)